# TX00DQ05-3001 Exercises 2

Note that you don't have to use the functions / other code in the cells below. They are there just in case you need inspiration to get started.

In [1]:
import numpy as np
print(np.__version__)
import numpy.linalg as LA
import copy as copy

1.15.4


## Exercise 1: Iterative policy evaluation. 

Calculate state-value function V for the gridworld of Sutton & Barto example 4.1. Policy is assumed to be random, ie. each of the four directions are equally likely. Movement that would result in leaving the grid (for example moving up in top row) will leave state unchanged (but action has been taken). Gamma (discount factor) is assumed to be = 1, ie. no discounting.

When norm of the difference between new V and the old one is less than eps, stop iteration.

Compare needed number of iterations between synchronous (sweep over all states, and update value function after the sweep) and asynchronous (use always the latest values) update of state-value function.

Note that numpy tensor assignment does not create a copy. You might want to use .copy() method to avoid sharing a reference to the same array.

In [168]:
rows_count = 4
columns_count = 4
V = np.zeros((rows_count, columns_count))
V_new = np.zeros((rows_count, columns_count))
terminating = [(0,0), (rows_count-1, columns_count-1)]
stepcost = -1


maxiters = 10000
eps = 0.0000001

directions = ["right","left","up","down"]

asyn = True

def valueWithBoundary(r,c,oR,oC):
    #print("-nstate: ",r,",",c)
    #print(V)
    if 0 > r or 4 <= r or 0 > c or 4 <= c :
        #print(oR,",",oC," value:",V[oR,oC])
        return V[oR,oC]
    else:
        #print(r,",",c," value:",V[r,c])
        return V[r,c]
    
def valueFor_String(row, column, direction):
# YOUR CODE HERE
    #print(direction)
    #print("-current: ",row,",",column)
    if direction == "right":
        return valueWithBoundary(row+1,column,row, column)
    elif direction ==  "left":
        return valueWithBoundary(row-1,column,row, column)
    elif direction ==  "up":
        return valueWithBoundary(row,column+1,row, column) 
    elif direction ==  "down":
         return valueWithBoundary(row,column-1,row, column)        
    
def valueFor_Number(row, column, direction):
# YOUR CODE HERE
    #print(direction)
    #print("-current: ",row,",",column)
    if direction == 0: 
        return valueWithBoundary(row+1,column,row, column)
    elif direction ==  1:
        return valueWithBoundary(row-1,column,row, column)
    elif direction ==  2:
        return valueWithBoundary(row,column+1,row, column) 
    elif direction ==  3:
        return valueWithBoundary(row,column-1,row, column)        
    

def calculateValueForCurrentState(value,stepcost= stepcost):
    #print("(",stepcost,"+",value,")*0.25 = ",((stepcost+(value))*0.25))
    return ((stepcost+(value))*0.25)
    

def updateStates(r,c):
    value = 0
    for d in directions:
       
        value += calculateValueForCurrentState(valueFor_String(r,c,d))
    return value
    
for i in range(maxiters):
# YOUR CODE HERE
    #print(V)
    for c in range(V.shape[0]):
        for r in range(V.shape[1]):
            if not (r, c) in terminating:
                if asyn:
                    V[r,c] = updateStates(r,c)
                else:
                    V_new[r,c] = updateStates(r,c)
                    
    if(abs(LA.norm(V)-LA.norm(V_new)) < eps):
        break
    if asyn:
        V_new = copy.copy(V)
    else:
        V = copy.copy(V_new)
        
print("iteration", i)
with np.printoptions(precision=3):
    print(V)

iteration 205
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]


In [3]:
print(V.shape)
print(terminating)

(4, 4)
[(0, 0), (3, 3)]


## Exercise 2: Greedy policy. 

Based on the state-value function computed in exercise 1, print out deterministic greedy policy function. Is the policy generated also optimal one?

In [166]:
# YOUR CODE HERE
def intToStringDirection(i):
    
    if i == 0:
        return "d"
    elif i ==1:
        return "u"
    elif i ==2:
        return "r"
    elif i ==3:
        return "l"
        

def greedyLogic(v1,v2):
    if v1 > v2:
        return 1
    elif v1==v2:
        return 0
    else:
        return -1
def gereedyPolicyFromTable(v,term):
    greedy = np.zeros((v.shape[0], v.shape[1]))
    test = []
    #test.append([])
    currentValue =-100000
    newValue= -100000
    d=0
    print(v)
    for row in range(v.shape[0]):
        test.append([])
        for column in range(v.shape[1]):
            if not (row, column) in term:
                currentValue = -100000
                
                for i in range(len(directions)):
                    newValue = valueFor_Number(row,column,i)
                    if(greedyLogic(newValue,currentValue)==1):
                        currentValue = newValue
                        d=i
                    elif (greedyLogic(newValue,currentValue)==0):
                        d=+i
                        
                test[row].append(intToStringDirection(d))
            else:
                test[row].append('0')
        
    return test

print(np.array(gereedyPolicyFromTable(V,terminating)))
print("0:down, 1:up, 2:right, 3:left")


[[  0.         -13.99999978 -19.99999969 -21.99999965]
 [-13.99999978 -17.99999973 -19.99999971 -19.99999971]
 [-19.99999969 -19.99999971 -17.99999975 -13.99999982]
 [-21.99999965 -19.99999971 -13.99999982   0.        ]]
[['0' 'l' 'l' 'l']
 ['l' 'l' 'l' 'd']
 ['l' 'r' 'd' 'd']
 ['r' 'r' 'r' '0']]
0:down, 1:up, 2:right, 3:left


## Exercise 3: Value function and policy in modified gridworld.

Change the definition of the exercise 1 gridworld by assigning a cost of -8 to movement in "up" direction. Compute the value function and greedy policy based on the value function. Is the greedy policy optimal?

In [167]:
rows_count = 4
columns_count = 4
V = np.zeros((rows_count, columns_count))
V_new = np.zeros((rows_count, columns_count))
terminating = [(0,0), (rows_count-1, columns_count-1)]
stepcost = -1


maxiters = 10000
eps = 0.0000001

directions = ["right","left","up","down"]

asyn = True

def valueWithBoundary(r,c,oR,oC):
    #print("-nstate: ",r,",",c)
    #print(V)
    if 0 > r or 4 <= r or 0 > c or 4 <= c :
        #print(oR,",",oC," value:",V[oR,oC])
        return V[oR,oC]
    else:
        #print(r,",",c," value:",V[r,c])
        return V[r,c]
    
def valueFor_String(row, column, direction):
# YOUR CODE HERE
    #print(direction)
    #print("-current: ",row,",",column)
    if direction == "right":
        return valueWithBoundary(row+1,column,row, column)
    elif direction ==  "left":
        return valueWithBoundary(row-1,column,row, column)
    elif direction ==  "up":
        return valueWithBoundary(row,column+1,row, column) 
    elif direction ==  "down":
         return valueWithBoundary(row,column-1,row, column)        
    
def valueFor_Number(row, column, direction):
# YOUR CODE HERE
    #print(direction)
    #print("-current: ",row,",",column)
    if direction == 0: 
        return valueWithBoundary(row+1,column,row, column)
    elif direction ==  1:
        return valueWithBoundary(row-8,column,row, column)
    elif direction ==  2:
        return valueWithBoundary(row,column+1,row, column) 
    elif direction ==  3:
        return valueWithBoundary(row,column-1,row, column)        
    

def calculateValueForCurrentState(value,stepcost= stepcost):
    #print("(",stepcost,"+",value,")*0.25 = ",((stepcost+(value))*0.25))
    return ((stepcost+(value))*0.25)
    

def updateStates(r,c):
    value = 0
    for d in directions:
       
        value += calculateValueForCurrentState(valueFor_String(r,c,d))
    return value
    
for i in range(maxiters):
# YOUR CODE HERE
    #print(V)
    for c in range(V.shape[0]):
        for r in range(V.shape[1]):
            if not (r, c) in terminating:
                if asyn:
                    V[r,c] = updateStates(r,c)
                else:
                    V_new[r,c] = updateStates(r,c)
                    
    if(abs(LA.norm(V)-LA.norm(V_new)) < eps):
        break
    if asyn:
        V_new = copy.copy(V)
    else:
        V = copy.copy(V_new)
        
print("iteration", i)
with np.printoptions(precision=3):
    print(V)
    print(np.array(gereedyPolicyFromTable(V,terminating)))
    print("0:down, 1:up, 2:right, 3:left")

iteration 205
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
[['0' 'l' 'l' 'l']
 ['l' 'l' 'l' 'd']
 ['l' 'r' 'd' 'd']
 ['r' 'r' 'r' '0']]
0:down, 1:up, 2:right, 3:left


## Extra exercise: Policy iteration

Implement policy iteration, ie. create a policy with the help of the value function from previous policy and iterate until policy is stable.

## Exercise 4: Value iteration

Solve the exercise 1 gridworld with value iteration algorithm. Solve also modified gridworld (cost of "up" movement = -4).

In [164]:
# rows_count = 4
columns_count = 4
V = np.zeros((rows_count, columns_count))
V_new = np.zeros((rows_count, columns_count))
terminating = [(0,0), (rows_count-1, columns_count-1)]
stepcost = -1


maxiters = 10
eps = 0.0000001

directions = ["right","left","up","down"]

asyn = False

       
    
def valueFor_Number(row, column, direction):
# YOUR CODE HERE
    #print(direction)
    #print("-current: ",row,",",column)
    if direction == 0: 
        return valueWithBoundary(row+1,column,row, column)
    elif direction ==  1:
        return valueWithBoundary(row-4,column,row, column)
    elif direction ==  2:
        return valueWithBoundary(row,column+1,row, column) 
    elif direction ==  3:
        return valueWithBoundary(row,column-1,row, column)        
    

def calculateValueForCurrentState(value,stepcost= stepcost):
    #print("(",stepcost,"+",value,")*0.25 = ",((stepcost+(value))*0.25))
    return ((stepcost+(value))*0.25)
    

def updateStates(r,c):
    value = 0
    for d in range(len(directions)):
       
        value += calculateValueForCurrentState(valueFor_Number(r,c,d))
    return value
    
for i in range(maxiters):
# YOUR CODE HERE
    #print(V)
    for c in range(V.shape[0]):
        for r in range(V.shape[1]):
            if not (r, c) in terminating:
                if asyn:
                    V[r,c] = updateStates(r,c)
                else:
                    V_new[r,c] = updateStates(r,c)
                    
    if(abs(LA.norm(V)-LA.norm(V_new)) < eps):
        break
    if asyn:
        V_new = copy.copy(V)
    else:
        V = copy.copy(V_new)
        
print("iteration", i)
with np.printoptions(precision=3):
    
    print(np.array(gereedyPolicyFromTable(V,terminating)))

iteration 9
[[ 0.    -6.61  -8.5   -8.849]
 [-9.688 -9.32  -8.594 -7.811]
 [-9.395 -8.676 -7.11  -5.042]
 [-9.173 -8.102 -5.392  0.   ]]
[['0' 'l' 'l' 'd']
 ['r' 'r' 'd' 'd']
 ['r' 'r' 'r' 'd']
 ['r' 'r' 'r' '0']]
